# PDAQP Algorithm Benchmarking

This notebook benchmarks the performance of the PDAQP algorithm implementation on CPU.

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import statistics  # For calculating standard deviation


In [ ]:
# Constants (based on the algorithm structure)
PDAQP_N_PARAMETER = 10  # Input parameter dimension
PDAQP_N_SOLUTION = 5   # Output solution dimension

# Define arrays from the C code implementation
# These are the coefficients for the halfplanes defining the decision regions
pdaqp_halfplanes = np.array([
    0.0263419286592053, -0.6386660079297741, 0.39258886470085763, -0.22019176717374056, -0.4915310232080545, -0.20733925940171036, 0.12525603863550702, 0.21276799769948984, 0.20214765840624327, -0.0488478757707278, 0.0, 0.3877149893360632, 0.33264693696021524, -0.24192511558287272, -0.03285849024879591, 0.19111181924585827, 0.07708312780960717, -0.4845294427900446, -0.16629470675640015, 0.048899457853142786, -0.6100297181216445, 0.0, 0.10899021897244414, 0.22649681103065192, -0.15033771383645456, -0.0013849164929540187, 0.026537114610993152, 0.026163042005418715, -0.43242819814367156, 0.23888452806746646, 0.5039720635416349, -0.64403451166607, 0.0, 0.4711526400108865, 0.09378922679864599, 0.3722765309055286, 0.3407957680752179, 0.34789580730962594, 0.36406460344225267, 0.02065666691402261, -0.16985583746187405, -0.41553437573733903, 0.24282946834307179, 0.0, 0.2054914265992108, -0.6471087254227091, 0.5596069045544073, -0.10699592980350005, -0.3942481559500167, -0.08450453086794457, 0.14161081114111912, 0.16340667027461078, 0.05958374129189267, 0.039209007902732074, 0.0, 0.36147149754631075, 0.32783804474441103, -0.2635522861298047, -0.05235025079834144, 0.17157911766601736, 0.056442850564447845, -0.4864951915466458, -0.15687673463298357, 0.07268024047324342, -0.624871770026171, 0.0, 0.31164861253943016, 0.28348721195052395, -0.009430985544227489, 0.13868145717761107, 0.17179375636336988, 0.17803995770190975, -0.4590842030080137, 0.18843725840786937, 0.3740195765408839, -0.5965063136312907, 0.0, 0.03618227723972094, -0.6209771880758927, 0.38072314864208906, -0.22119216268280084, -0.49111763675666925, -0.2058277825476645, 0.08712842190100836, 0.23495330923535807, 0.2479694281774716, -0.10657025862149283, 0.0, 0.4616047003504276, 0.24596866751799695, -0.1951221656134922, -0.04798131707321298, 0.25821253013335793, 0.08670171909161037, -0.24258768781705284, -0.5203169901189648, -0.4949930869942629, -0.19302341164151607, 0.0, 0.5542840423567839, 0.19457803114334346, 0.3407012394319408, 0.3679298648375163, 0.38709219792819977, 0.4044215103477245, -0.15553408419783657, -0.08540368104056924, -0.24201575517551707, -0.0023402202151552977, 0.0, 0.22841549743609446, -0.5471629953692986, 0.31714000117507984, -0.26430225712439054, -0.4543879247422333, -0.1934827229202361, -0.10769305648558282, 0.15383772216730593, 0.2522496361535353, -0.36770624855519657, 0.0, 0.2731938238324235, 0.033873529978354265, -0.04632851521491493, -0.034992061526719874, 0.17565364819658377, 0.04753824526066772, 0.10523387879062142, -0.5478518401951987, -0.7047090151586591, 0.2826698602805244, 0.0, 0.44980389431713597, 0.0749681110829066, 0.3866805660153155, 0.3432238683517712, 0.33756060121472947, 0.36025966021116357, 0.04832689142119228, -0.160646775610184, -0.41899889883998837, 0.278019101128144, 0.0, 0.44922451170527855, 0.046336225351369664, -0.07056568863622763, -0.14983642497502866, -0.03727416722882488, -0.019692736606214553, -0.48028897057595155, -0.07778013509663803, 0.15861073166624218, -0.7106040003767469, 0.0, 0.11177713468004304, 0.17035024003474755, -0.11586857334552408, -0.021058895419998166, -0.017262869547860195, 0.007746785267584113, -0.4229615007142653, 0.2588409848711415, 0.5240353073345881, -0.6509620229334389, 0.0, 0.5133396917830506, -0.14018508127335202, 0.545548985087528, 0.28125694185908034, 0.18670927150252362, 0.310957359551938, 0.0692188178642795, -0.10140278168503988, -0.36790672720305906, 0.2410754674966696, 0.0, 0.29576090133446886, -0.5941310063939332, 0.5766408759778011, -0.07375857676452582, -0.3622168916435647, -0.039987809677739454, 0.0241574777017671, 0.2193556468874341, 0.16138552747416735, -0.11846069156137698, 0.0, 0.18405502937056745, 0.1656652490057719, -0.46345549804724284, -0.3045463111523128, 0.05128788510678158, -0.16685026407543688, -0.18778118253954332, -0.5692304132904098, -0.43369759216068254, -0.2308489158975905, 0.0, 0.42320162635726366, -0.558675422009294, 0.49366669431780713, -0.14862836531493243, -0.35523431412281536, -0.06575990014103344, -0.09670193461641789, 0.10196041323592261, 0.1057474439539606, -0.2855511616348011, 0.0, 0.01842491655482211, 0.01822630801480811, 0.3813399247421765, 0.33051542568128406, 0.05200143643117614, 0.23738457490020973, -0.09665670802555916, 0.5787240606012538, 0.5680684820316466, -0.13655143989027585, 0.0, 0.3820616896731582, -0.5893497678604467, 0.32895327800577306, -0.30899333146155, -0.41957833133891204, -0.1915648560469231, -0.06933096128956619, -0.0891664056571796, -0.05454464224449239, -0.27300725286760885, 0.0, 0.35781021559967424, 0.06931810603676902, 0.5411059707590423, 0.4752037779027177, 0.29291142534862424, 0.4288106523125714, -0.024573125746406622, 0.24614485788417573, 0.04058419270305487, 0.1266438421648084, 0.0, 0.5172374289755595, 0.029091149683788682, -0.002591635049877471, -0.11568410927275068, -0.014113786546579871, 0.01921310935919493, -0.4753684960563831, -0.09112898622207438, 0.11364294242318863, -0.6859058662010886, 0.0, 0.37732300085499926, 0.12085920813051194, 0.13944090565262246, 0.11499888765923025, 0.07268123251542685, 0.1617288921010522, -0.43736513342643846, 0.23858701308572047, 0.4029711039044669, -0.6068899118219777, 0.0, 0.5981144402065366, -0.14906733088296772, 0.036507592802802184, -0.22102741308585735, -0.03902756112893308, -0.04300240525337997, -0.23677881130574796, -0.47273391289430916, -0.43186158531784014, -0.3169128925587501, 0.0, 0.6272130023751774, -0.07238509423578159, 0.5510371471092109, 0.3031282468131187, 0.1996118897555325, 0.35041503934530976, -0.13108421622207167, 0.014409438484383953, -0.1520791694466853, -0.05178023274383887, 0.0, 0.4104051185866229, -0.22156990447412142, 0.09888788227909938, -0.16384639606795887, -0.020779643793210317, -0.03900532892906653, 0.06544583168086195, -0.5333162652010942, -0.6602726950762754, 0.13866262851380062, 0.0, 0.5738026884252441, -0.13546625651689911, 0.5409721757397127, 0.2646456640491174, 0.1834943795202725, 0.3109273531338233, 0.009431760245161961, -0.1119699352681885, -0.35087421575469613, 0.15370816389382286, 0.0, 0.4343675089723924, -0.5853509873409257, 0.40575286352037154, -0.24707611312384517, -0.38353182543818687, -0.13488554251482845, -0.07332162657410515, -0.05643918000351393, -0.04951704125704251, -0.258269136358747, 0.0, 0.36752807221142303, -0.13072528473235356, -0.21659102424810586, -0.38516845489039897, -0.1361103639435271, -0.210793966296032, -0.20013557971049306, -0.5282913417005598, -0.4055863025984401, -0.32551687729477435, 0.0, 0.09957406562805472, -0.17564096802531637, -0.25758244709572076, -0.3849631960780278, -0.15365347861073383, -0.264992796703916, 0.07326196858158547, -0.5724604736962966, -0.5603313468485146, 0.061723369794183455, 0.0, 0.4131974735770089, -0.010374606778534499, 0.5909490680563086, 0.43747954579722725, 0.2384277400814437, 0.40666397485294686, -0.034250799247476454, 0.23626684815502697, 0.03352455835140136, 0.09060159410186333, 0.0
], dtype=np.float64)

# Feedback coefficients for the affine functions at the leaf nodes
pdaqp_feedbacks = np.array([
    0.1841987895149287, -0.08369850727930518, -0.053393240070157826, -0.17845927650728752, 0.005111402919304886, -0.11138109474751055, -0.27966586978829655, -0.03430340432434822, 0.014012162918987587, -0.2013106545183362, 0.0, 0.17516104303155747, 0.135423690205078, -0.18808795356065466, -0.102912898016832, 0.17375253446842304, -0.04024432891001396, -0.3226404106202411, -0.1073027354419698, -0.05534339807433291, -0.18455126293683438, 0.0, -0.12898753098415708, -0.12552596199334212, 0.0016936671541160633, -0.07713658497109185, 0.02383213177579081, -0.10071329190516264, 0.05745564767635871, 0.023149527959866894, -0.09370327776218297, 0.2939952830654985, 0.0, -0.05898194703951358, 0.019955363440989877, -0.09486985500391465, -0.07802613178083556, 0.040877204473049836, -0.08390849022548322, -0.22029761100214784, 0.17658761699934433, 0.2300033713368373, -0.11967497524039711, 0.0, 0.25503776926042365, 0.08246518269206045, 0.15324967278055654, 0.1491117027007601, 0.2727471622747581, 0.1587378494465609, -0.20653009681760753, 0.06337996912873665, -0.046947186881701894, 0.04216905211896412, 0.0, 0.2290469586363777, -0.07477088042394317, -0.017956909281124568, -0.14601954284066943, 0.03822697701923413, -0.07672644246622048, -0.27769959868312855, -0.050471674933649194, -0.02554180434656098, -0.17819615616922266, 0.0, 0.1630637164323518, 0.13301555628717307, -0.19764653494839698, -0.11166317877115613, 0.1648199531968869, -0.04959206096013105, -0.32317079164280943, -0.10294151271169809, -0.04467412738101897, -0.1907861582196528, 0.0, -0.12003763559344502, -0.12374436607177121, 0.00876533722564581, -0.07066291497728161, 0.030440671995801367, -0.09379761305329178, 0.057848036488280165, 0.019922989788456448, -0.10159666335211781, 0.29860800991860925, 0.0, 0.0650884513589141, 0.044653233594541895, 0.0031631257986892212, 0.011716901244032215, 0.13248991745069577, 0.01196201280034227, -0.2148580132432247, 0.13185883971795423, 0.1205791394100399, -0.05572977704205963, 0.0, 0.0650884516225609, 0.04465323367640237, 0.0031631256317477476, 0.011716900939685275, 0.1324899175668885, 0.01196201290660458, -0.21485801310175215, 0.13185883941428753, 0.12057913994743812, -0.055729777247408005, 0.0, 0.15628990372947718, -0.14169704753461354, -0.014896592319942802, -0.17810464396959685, -0.0016838977075230434, -0.11808060742047867, -0.1689349315137722, -0.09547404050888914, -0.11503885556757106, -0.03639415276427121, 0.0, 0.1439252268290917, 0.07051132592405308, -0.14500224136682108, -0.1025159909941537, 0.1661471873799596, -0.047742469479899456, -0.19870958510965328, -0.17576533050864, -0.19977818286610005, 2.4417010045130553e-05, 0.0, -0.09623354493379441, -0.05745863001170439, -0.04348616137542491, -0.07755278310452632, 0.03180712533479461, -0.09285071551343871, -0.07249864918630111, 0.0949396451627564, 0.057751542820869146, 0.10044857973380841, 0.0, -0.09623354486683536, -0.05745862976805436, -0.043486161521629346, -0.07755278336224655, 0.03180712524796466, -0.0928507155964805, -0.07249864929836702, 0.09493964513114576, 0.057751543421955646, 0.10044857978672825, 0.0, 0.24541921261644167, 0.0624764860984856, 0.1665172102588871, 0.14923392391850218, 0.27040522053251637, 0.15642892007839676, -0.16836763465956775, 0.042298037342384, -0.09142350670767967, 0.09900610124283236, 0.0, 0.06034385417135251, -0.18996204096205818, 0.02388935197416729, -0.16796268381897053, -0.05593896319853312, -0.1360051768978198, -0.12488371161202917, 0.018819174690891057, -0.0016086910737994442, -0.006437633749262721, 0.0, -0.009390424354827868, -0.022915507878836432, -0.07293213930795706, -0.08727233014978596, 0.08278373601374067, -0.07693572544240251, -0.09200548009764256, -0.028146826921591598, -0.07861943067114066, 0.10582151825187773, 0.0, -0.009390424937115146, -0.02291550780394282, -0.0729321394686141, -0.08727233041391115, 0.08278373603948541, -0.07693572550915323, -0.0920054795983821, -0.028146826751900774, -0.07861943055463758, 0.10582151843586891, 0.0, -0.125768473496435, -0.03734531910497999, -0.053196619320875965, -0.07236603491777774, 0.007956902570602964, -0.09718657968939609, -0.13683414135685992, 0.20523300358297974, 0.2215801096058141, -0.014593234955306136, 0.0, 0.17353598778355864, 0.012539286970110666, 0.20410488446993189, 0.1560189042585369, 0.23257344138242345, 0.14253416290582754, -0.1046768960502611, 0.09833687079399656, -0.05722636893755174, 0.17040373439978268, 0.0, 0.18241912730485788, -0.04055019028003954, -0.07991656913304665, -0.16358310600126066, 0.038319270608906456, -0.09737324053555985, -0.2881281757069886, -0.04867802418763106, 0.0003550539213219636, -0.19801048891344247, 0.0, 0.18241912678018557, -0.04055019069943882, -0.07991656874423883, -0.16358310672139845, 0.038319270575459294, -0.097373240719137, -0.2881281754892185, -0.04867802375932868, 0.0003550536976208932, -0.19801048919367464, 0.0, -0.13115831547846596, -0.07289480824456827, -0.030658775194742565, -0.05899103379630807, 0.06433819365993262, -0.08362688936199515, 0.047133554471155366, 0.00561575423456033, -0.11036185106152631, 0.2980207360672282, 0.0, -0.05964738531851918, 0.03608905675464948, -0.10478725870881578, -0.07246374633645655, 0.05329404020104389, -0.07867077906656694, -0.2234617728069265, 0.1712127671068865, 0.2248968080763828, -0.11844100214885422, 0.0, 0.260557165360205, -0.05135382317821634, 0.23550839844122542, 0.10297515735254907, 0.1697571920517593, 0.11529427608423451, -0.18028533260735985, 0.10796102027941368, -0.004591404067575719, 0.03193400889485468, 0.0, 0.15495864094095219, -0.14216437954288313, -0.015714878350298965, -0.1789883270697269, -0.0026136039474700867, -0.11905193476856833, -0.16856137410102184, -0.09526892025114697, -0.11445758927270164, -0.03638853244070137, 0.0, 0.06319541021444208, 0.042171616335606135, -0.19462435980483203, -0.15610387778842655, 0.10976836380256755, -0.10664525597736074, -0.1760565101550094, -0.16332653957975635, -0.16452931033248325, 0.00036526324767342496, 0.0, -0.016331698061303145, -0.029409573768502277, 0.005627029992925667, -0.024514496769994287, 0.0876077241395261, -0.03455203985025738, -0.09491939349152279, 0.08262842692114157, 0.022864184764829692, 0.10011122952619426, 0.0, -0.016331697850858928, -0.029409573811991822, 0.005627029621842805, -0.024514497065098548, 0.0876077240386801, -0.03455203989429695, -0.09491939325108655, 0.08262842717570744, 0.022864185440561013, 0.10011122948687731, 0.0, -0.016331698475966095, -0.029409574295440007, 0.005627029063310918, -0.024514497217609024, 0.08760772398134599, -0.034552040194139655, -0.09491939318376132, 0.08262842638893493, 0.022864185805225534, 0.1001112292508213, 0.0, 0.11142522564071451, -0.18144838878797934, 0.06780220835389164, -0.128984929106907, -0.017604349943377787, -0.0950927754082939, -0.1193955335987758, 0.0005755407180434449, -0.04919173475832881, 0.025135229915423968, 0.0, -0.008746186040309476, -0.022808133886651878, -0.07237831080560005, -0.08678074253174807, 0.0832672120832702, -0.07641973796244181, -0.09193626276481422, -0.02837691571887459, -0.07921954819304759, 0.10621971564526542, 0.0, -0.008746186269479586, -0.022808133595166917, -0.07237831029347712, -0.0867807426032484, 0.08326721250155439, -0.07641973772724689, -0.09193626258258251, -0.0283769154399832, -0.07921954810315852, 0.10621971546513156, 0.0, -0.002707985069638686, -0.01683502002332603, 0.052594148755440794, 0.021535546324930425, 0.10030908796159349, 0.001375772417173585, -0.12361253307686515, 0.1612821377747667, 0.10694746643894214, 0.06146917082543897, 0.0, -0.0027079846914498046, -0.016835020056997784, 0.05259414835237495, 0.021535546247132782, 0.10030908771170505, 0.0013757728649071844, -0.12361253310440712, 0.16128213722920298, 0.10694746655753952, 0.06146917110134435, 0.0, 0.06733981052309537, -0.189094606959479, 0.022702969945586172, -0.16885876198863128, -0.05144081906757228, -0.13478781593267947, -0.12218887900519831, 0.004789767148107725, -0.01965490217151421, 0.0008009850590876165, 0.0, -0.03808383213591338, -0.02647322753544992, -0.06806627927788271, -0.08359713196007303, 0.06433492362299649, -0.08192864313771704, -0.10305814000768004, 0.029393770682835947, -0.004604204782377453, 0.07613284781883979, 0.0, -0.038083832016304234, -0.026473227514280932, -0.06806627922381706, -0.08359713185649334, 0.06433492313463897, -0.08192864278889629, -0.10305813967313686, 0.029393771017372068, -0.00460420455042397, 0.07613284830073304, 0.0, -0.038083831543013355, -0.026473227214672608, -0.06806627935108298, -0.08359713215310158, 0.06433492355189561, -0.08192864293023576, -0.10305813979759744, 0.02939377044659907, -0.004604204099586854, 0.07613284778711908, 0.0, 0.1450820409228273, 0.009011258253999808, 0.20893013660907622, 0.1596634312601274, 0.21427859310487293, 0.13758291362847672, -0.1156373167070843, 0.15539726491943118, 0.016171165554702986, 0.14096283010098948, 0.0, 0.22157568031912345, -0.051243237054784854, -0.03830315292760098, -0.14212937367665274, 0.0525610910873771, -0.07365401813624416, -0.28284829900422453, -0.05641283101677249, -0.027708155786421485, -0.179621721021001, 0.0, 0.22157568029137653, -0.05124323505641047, -0.038303153785473464, -0.1421293739131217, 0.0525610909652054, -0.07365401843233045, -0.2828482987935716, -0.05641283088088681, -0.027708155816618018, -0.17962172142895055, 0.0, -0.13666237956072855, -0.07139173425514435, -0.03650819005252233, -0.062006690496429935, 0.06233628378552346, -0.08696099611453222, 0.046391385655397584, 0.00670300197535315, -0.10641712954950135, 0.29543590796095826, 0.0, 0.0799103137757994, -0.0020219791053191966, 0.04352693222369982, 0.003999406743680609, 0.10405324821377648, 0.005866797429986226, -0.20464378712015835, 0.14364517583447942, 0.1248768466720843, -0.05290167617889695, 0.0, 0.07991031364534128, -0.0020219787534928673, 0.043526931573637, 0.003999406782092976, 0.10405324862321363, 0.005866797735437823, -0.2046437874644375, 0.1436451755144497, 0.12487684691226843, -0.05290167601966657, 0.0, 0.1530080942118666, -0.08537310095173381, -0.049428988615213304, -0.1580420462705722, 0.0428615122564475, -0.09941159136008254, -0.17683766361541528, -0.11678480356074579, -0.13753020837475122, -0.026728004315756077, 0.0, 0.1530080945432992, -0.0853731006975744, -0.04942898834666762, -0.15804204574669098, 0.042861513002183015, -0.09941159101788172, -0.17683766399760317, -0.11678480381071858, -0.13753020852300762, -0.026728004467785543, 0.0, -0.0982558492208137, -0.022750894007985967, -0.06476558803697302, -0.0651898829374358, 0.059256734309694456, -0.08134656314348894, -0.07736844180540321, 0.08180760494573022, 0.04389200418380527, 0.10640501827402653, 0.0, -0.09825584908386181, -0.02275089378956191, -0.0647655884327814, -0.06518988292458114, 0.05925673446696046, -0.08134656324556937, -0.07736844213290932, 0.08180760468682294, 0.043892004404836614, 0.10640501775377749, 0.0, 0.2527454058340516, -0.06325907675769998, 0.24360609609120487, 0.10444689848394337, 0.17096364039413214, 0.11475287726886599, -0.15072585710603004, 0.0898714218123536, -0.041214616306879295, 0.07742773801421406, 0.0, 0.0055855622255933535, -0.05879006785477827, -0.0521389603096647, -0.10460122016590702, 0.05299194185754702, -0.08962135590170502, -0.09906633965555205, -0.018060507739955798, -0.062080766506973105, 0.08171297936639592, 0.0, 0.005585561934600565, -0.058790068574451536, -0.052138961021375246, -0.10460122008631366, 0.052991942180242536, -0.08962135603687357, -0.09906633909216973, -0.018060507634319458, -0.062080767212836695, 0.08171298060309307, 0.0, 0.005585561078299788, -0.05879006686576952, -0.05213896090648476, -0.10460121984537331, 0.05299194353791777, -0.08962135586298672, -0.0990663388428941, -0.01806050786083555, -0.06208076749701922, 0.08171298160474583, 0.0, -0.15242886557952035, 0.026518912011333672, -0.0902128331427752, -0.041516981691046055, 0.06099253488181405, -0.0746035006460406, -0.12426433242780245, 0.18727724277121272, 0.19213782424852013, 0.028325012015433793, 0.0, 0.20269973078729087, -0.057321652910090025, 0.24459683796601966, 0.12227319400071628, 0.17455788183767526, 0.11783058328361544, -0.11842698146844119, 0.11797863733310589, -0.025019519106546522, 0.12345556043474251, 0.0, 0.1125495518687958, -0.18033618273468, 0.09107234598434345, -0.10881620572185516, -0.014431115799751676, -0.08060708622816593, -0.12529372250612084, 0.035890459331306176, -0.014527041370807648, 0.01680258355828966, 0.0, -0.006208825027506157, -0.02029812380725448, -0.019862634969168423, -0.04126428876423479, 0.09042851506323939, -0.04372866499570607, -0.10524720135591702, 0.05132123020872198, -0.000988820130626601, 0.08741473192745375, 0.0, -0.00620882534158377, -0.020298123720761844, -0.019862634813680537, -0.041264289319808835, 0.09042851468778755, -0.043728664903172794, -0.10524720131581956, 0.05132123033891018, -0.0009888199414072138, 0.08741473189817016, 0.0, -0.006208825293480249, -0.020298123663193807, -0.01986263527623358, -0.04126428981537259, 0.09042851464524741, -0.043728665341954874, -0.10524720109529068, 0.05132123039731862, -0.0009888195551593533, 0.08741473180009716, 0.0, -0.006208825022430329, -0.020298124220319763, -0.01986263577737865, -0.041264290397593016, 0.0904285146325839, -0.04372866532266493, -0.10524720130783766, 0.051321228931899054, -0.0009888194266113707, 0.08741473167435533, 0.0, 0.12508342251448626, -0.08215038391107943, -0.07396217087489837, -0.17153787055045513, 0.03397442549061567, -0.11501271084247407, -0.17100155451223537, -0.11742633804682205, -0.13075936503075894, -0.024422653716581998, 0.0, 0.12508342206211095, -0.08215038332810376, -0.07396217178347236, -0.17153787082301306, 0.03397442624378413, -0.11501271066541534, -0.17100155433402114, -0.11742633800019016, -0.13075936516913686, -0.02442265360284305, 0.0, -0.01485500986909829, -0.03237597796951522, 0.008506108114983622, -0.024882761742135967, 0.08579923145323541, -0.034751693117526995, -0.09479877825516067, 0.08372363543800584, 0.023669957360208316, 0.09951977405762853, 0.0, -0.014855009884706704, -0.0323759776584223, 0.008506107275202518, -0.024882762340233393, 0.08579923122407969, -0.03475169341007304, -0.0947987786837539, 0.08372363554302066, 0.02366995836082238, 0.09951977296941181, 0.0, -0.01485501051960233, -0.03237597723632342, 0.008506107478574271, -0.02488276257984274, 0.08579923174591635, -0.034751692629213814, -0.09479877884791583, 0.08372363507148183, 0.023669958450502276, 0.09951977354149136, 0.0, 0.0213252906807511, -0.06250598304408679, -0.03729979241159012, -0.09734184148054177, 0.05802529528182275, -0.08109244484625522, -0.09880762135789724, -0.021131905487283047, -0.07170544268014557, 0.08592928080624622, 0.0, 0.021325289917840537, -0.0625059834445887, -0.03729979192828392, -0.09734184137570602, 0.05802529627787348, -0.08109244514292815, -0.09880762063478798, -0.02113190578318434, -0.07170544267867172, 0.08592928120221868, 0.0, 0.021325288858533922, -0.06250598243523252, -0.037299792735417614, -0.09734184161269176, 0.058025296665919684, -0.0810924450566409, -0.09880762029858559, -0.02113190555457724, -0.07170544291145688, 0.08592928165283129, 0.0, -0.010094362666043006, -0.007084141413962481, 0.04397790400851748, 0.02412964176792878, 0.10650919347443573, 0.002523517291342615, -0.12192473926091775, 0.1595025646795693, 0.10510179623735422, 0.06645305770896208, 0.0, -0.010094363120240606, -0.0070841409747470045, 0.043977902861496, 0.024129641415213763, 0.10650919361597903, 0.002523517530130447, -0.12192473948840775, 0.15950256436437488, 0.10510179666096992, 0.06645305749746207, 0.0, -0.029366542016810143, -0.0399200925383659, -0.060560736076669404, -0.09064726056211339, 0.05476163800296046, -0.0862994718938059, -0.1046400266128595, 0.027359309802989316, -0.0058487199884850165, 0.06990379307994446, 0.0, -0.02936654378348241, -0.03992009310544449, -0.060560735786254205, -0.09064726030449498, 0.05476163849911553, -0.08629947177403961, -0.10464002568120323, 0.02735931048707301, -0.005848719641652349, 0.06990379385491992, 0.0, -0.02936654311338087, -0.03992009109581859, -0.06056073623594638, -0.09064725994776979, 0.054761639198610476, -0.0862994714919141, -0.10464002546161161, 0.02735931064627731, -0.005848719238920075, 0.06990379391967208, 0.0, -0.029366543919652502, -0.039920090998084835, -0.06056073638492454, -0.0906472599721323, 0.05476163918597539, -0.08629947109849766, -0.10464002559654667, 0.027359311360198615, -0.005848717909621137, 0.06990379450113623, 0.0, 0.180214440539177, -0.045182273099958425, 0.23917896521098614, 0.13125001587868562, 0.17569635829854802, 0.11996760841114838, -0.1220126301106784, 0.14719798807593173, 0.011155524587211849, 0.11585850713931402, 0.0, 0.009068025101351268, -0.040885112454156214, -0.005592171891962344, -0.04995403815961345, 0.07693957291912773, -0.04847263426029413, -0.1078259474500891, 0.04933624428725129, -0.0027303518585257073, 0.07833132050980358, 0.0, 0.009068023694831069, -0.040885112416406494, -0.0055921721015367515, -0.04995403803726633, 0.07693957366318084, -0.04847263414477109, -0.10782594716752698, 0.04933624532291495, -0.0027303509536834645, 0.0783313210369816, 0.0, 0.009068022776806876, -0.04088511103028525, -0.0055921734380155646, -0.04995403739573912, 0.07693957468970115, -0.048472633691933546, -0.10782594670282612, 0.049336245656751106, -0.002730350736067253, 0.07833132136352454, 0.0, 0.0090680229302855, -0.0408851102876425, -0.005592173491210465, -0.04995403775941971, 0.07693957464491137, -0.04847263370463289, -0.10782594710591686, 0.04933624605204215, -0.0027303499534020617, 0.07833132145963226, 0.0, 0.009068022965231372, -0.040885110192751116, -0.00559217383658814, -0.04995403753596899, 0.07693957424069635, -0.04847263416452669, -0.10782594711516547, 0.04933624567907999, -0.002730349368300061, 0.07833132167077075, 0.0
], dtype=np.float64)

# Indices for halfplanes and jumps in the decision tree
pdaqp_hp_list = np.array([
    4, 23, 18, 10, 7, 20, 28, 13, 7, 16, 20, 27, 10, 31, 19, 27, 14, 0, 11, 24, 28, 28, 21, 31, 30, 27, 11, 20, 15, 6, 21, 31, 22, 15, 24, 14, 1, 20, 1, 31, 29, 15, 17, 31, 11, 9, 17, 26, 15, 15, 13, 10, 13, 7, 3, 14, 7, 11, 9, 9, 17, 26, 15, 30, 8, 4, 8, 14, 9, 15, 24, 3, 0, 14, 8, 3, 2, 14, 15, 15, 25, 11, 5, 14, 15, 8, 5, 11, 5, 14, 10, 8, 2, 8, 5, 8, 5, 11, 5, 14, 10, 15, 13, 4, 8, 14, 9, 7, 2, 0, 2, 9, 12, 7, 2, 7, 2, 3, 2, 7, 2, 7, 2, 12, 5, 1, 6, 6, 1, 3, 0
], dtype=np.int32)

pdaqp_jump_list = np.array([
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 0, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 0, 39, 40, 0, 40, 41, 42, 0, 0, 0, 0, 0, 0, 0, 0, 0, 34, 35, 36, 37, 38, 0, 38, 0, 0, 0, 0, 0, 34, 35, 0, 0, 0, 33, 0, 33, 0, 0, 0, 31, 0, 0, 0, 0, 28, 0, 0, 0, 0, 0, 24, 25, 26, 0, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 2, 3, 4, 0, 0, 0, 0, 0
], dtype=np.int32)


In [ ]:
def pdaqp_evaluate(parameter, solution):
    """
    Standard implementation of the PDAQP algorithm.
    Direct translation of the C function to Python.
    
    Args:
        parameter: Input vector of dimension PDAQP_N_PARAMETER
        solution: Output vector of dimension PDAQP_N_SOLUTION
    
    Returns:
        None (solution is modified in-place)
    """
    id = 0
    next_id = id + pdaqp_jump_list[id]
    
    # Traverse the decision tree until a leaf node is reached
    while next_id != id:
        # Compute halfplane value
        disp = pdaqp_hp_list[id] * (PDAQP_N_PARAMETER + 1)
        val = 0
        for i in range(PDAQP_N_PARAMETER):
            val += parameter[i] * pdaqp_halfplanes[disp + i]
        
        # Determine which branch to take based on halfplane evaluation
        id = next_id + (1 if val <= pdaqp_halfplanes[disp + PDAQP_N_PARAMETER] else 0)
        next_id = id + pdaqp_jump_list[id]
    
    # Leaf node reached -> evaluate affine function
    disp = pdaqp_hp_list[id] * (PDAQP_N_PARAMETER + 1) * PDAQP_N_SOLUTION
    for i in range(PDAQP_N_SOLUTION):
        val = 0
        for j in range(PDAQP_N_PARAMETER):
            val += parameter[j] * pdaqp_feedbacks[disp + j]
        val += pdaqp_feedbacks[disp + PDAQP_N_PARAMETER]
        solution[i] = val
        disp += PDAQP_N_PARAMETER + 1

def pdaqp_evaluate_vectorized(parameter, solution):
    """
    Optimized version of the PDAQP algorithm using NumPy vectorization.
    
    Args:
        parameter: Input vector of dimension PDAQP_N_PARAMETER
        solution: Output vector of dimension PDAQP_N_SOLUTION
    
    Returns:
        None (solution is modified in-place)
    """
    id = 0
    next_id = id + pdaqp_jump_list[id]
    
    # Traverse the decision tree until a leaf node is reached
    while next_id != id:
        # Compute halfplane value using vectorized dot product
        disp = pdaqp_hp_list[id] * (PDAQP_N_PARAMETER + 1)
        coeffs = pdaqp_halfplanes[disp:disp+PDAQP_N_PARAMETER]
        constant = pdaqp_halfplanes[disp+PDAQP_N_PARAMETER]
        val = np.dot(parameter, coeffs)
        
        # Determine which branch to take based on halfplane evaluation
        id = next_id + (1 if val <= constant else 0)
        next_id = id + pdaqp_jump_list[id]
    
    # Leaf node reached -> evaluate affine function using vectorized operations
    disp = pdaqp_hp_list[id] * (PDAQP_N_PARAMETER + 1) * PDAQP_N_SOLUTION
    
    for i in range(PDAQP_N_SOLUTION):
        offset = disp + i*(PDAQP_N_PARAMETER + 1)
        coeffs = pdaqp_feedbacks[offset:offset+PDAQP_N_PARAMETER]
        constant = pdaqp_feedbacks[offset+PDAQP_N_PARAMETER]
        solution[i] = np.dot(parameter, coeffs) + constant


In [ ]:
def test_performance(n_tests=10000, use_vectorized=False):
    """
    Test CPU performance of the algorithm.
    
    Args:
        n_tests: Number of random inputs to test
        use_vectorized: Whether to use the vectorized implementation
    
    Returns:
        avg_time: Average execution time per evaluation (seconds)
        total_time: Total execution time for all evaluations (seconds)
        solutions: Array of computed solutions
    """
    # Generate random parameter vectors
    parameters = np.random.randn(n_tests, PDAQP_N_PARAMETER)
    solutions = np.zeros((n_tests, PDAQP_N_SOLUTION))
    
    # Select implementation
    evaluate_fn = pdaqp_evaluate_vectorized if use_vectorized else pdaqp_evaluate
    
    # Warm up to avoid measuring JIT compilation time
    for i in range(10):
        evaluate_fn(parameters[i % 10], solutions[i % 10])
    
    # Measure performance
    start_time = time.time()
    for i in range(n_tests):
        evaluate_fn(parameters[i], solutions[i])
    end_time = time.time()
    
    total_time = end_time - start_time
    avg_time = total_time / n_tests
    
    return avg_time, total_time, solutions

def benchmark_and_compare():
    """
    Run benchmarks and compare standard vs vectorized implementations.
    
    Returns:
        std_avg_time: Average execution time for standard implementation
        vec_avg_time: Average execution time for vectorized implementation
    """
    n_tests = 10000
    print(f"Running performance tests with {n_tests} random inputs...")
    
    # Test standard implementation
    std_avg_time, std_total_time, std_solutions = test_performance(n_tests, use_vectorized=False)
    print(f"Standard implementation:")
    print(f"  Average time: {std_avg_time * 1e6:.2f} microseconds per evaluation")
    print(f"  Total time: {std_total_time:.4f} seconds for {n_tests} evaluations")
    
    # Test vectorized implementation
    vec_avg_time, vec_total_time, vec_solutions = test_performance(n_tests, use_vectorized=True)
    print(f"Vectorized implementation:")
    print(f"  Average time: {vec_avg_time * 1e6:.2f} microseconds per evaluation")
    print(f"  Total time: {vec_total_time:.4f} seconds for {n_tests} evaluations")
    
    # Calculate speedup
    speedup = std_avg_time / vec_avg_time
    print(f"Speedup: {speedup:.2f}x")
    
    # Verify both implementations give the same results
    if np.allclose(std_solutions, vec_solutions):
        print("Both implementations produce identical results ✓")
    else:
        print("WARNING: Results differ between implementations!")
    
    return std_avg_time, vec_avg_time


In [ ]:
def extended_benchmark():
    """
    Extended benchmark with detailed timing metrics.
    Measures execution time, throughput, and latency jitter.
    
    Returns:
        metrics: Dictionary containing measured performance metrics
    """
    n_tests = 10000
    print(f"Running extended performance tests with {n_tests} random inputs...")
    
    # Generate random parameter vectors
    parameters = np.random.randn(n_tests, PDAQP_N_PARAMETER)
    std_solutions = np.zeros((n_tests, PDAQP_N_SOLUTION))
    vec_solutions = np.zeros((n_tests, PDAQP_N_SOLUTION))
    
    # Measure initialization overhead
    start_time = time.time()
    # Simulate initialization process
    dummy = np.zeros_like(pdaqp_halfplanes)
    dummy2 = np.zeros_like(pdaqp_feedbacks)
    init_time = (time.time() - start_time) * 1e6  # microseconds
    
    # Warm up phase
    for i in range(100):
        pdaqp_evaluate(parameters[i % 100], std_solutions[i % 100])
        pdaqp_evaluate_vectorized(parameters[i % 100], vec_solutions[i % 100])
    
    # Measure standard implementation with detailed timing
    std_times = []
    start_time = time.time()
    
    for i in range(n_tests):
        iter_start = time.time()
        pdaqp_evaluate(parameters[i], std_solutions[i])
        std_times.append((time.time() - iter_start) * 1e6)  # microseconds
    
    std_total_time = time.time() - start_time
    
    # Measure vectorized implementation
    vec_times = []
    start_time = time.time()
    
    for i in range(n_tests):
        iter_start = time.time()
        pdaqp_evaluate_vectorized(parameters[i], vec_solutions[i])
        vec_times.append((time.time() - iter_start) * 1e6)  # microseconds
    
    vec_total_time = time.time() - start_time
    
    # Calculate performance metrics
    std_avg_time = sum(std_times) / n_tests
    vec_avg_time = sum(vec_times) / n_tests
    
    # Calculate throughput in samples/second and convert to KOPS
    std_throughput = 1e6 / std_avg_time  # samples per second
    std_throughput_kops = std_throughput / 1e3  # convert to KOPS (thousands of samples/s)
    
    vec_throughput = 1e6 / vec_avg_time
    vec_throughput_kops = vec_throughput / 1e3
    
    std_jitter = statistics.stdev(std_times)
    vec_jitter = statistics.stdev(vec_times)
    
    # Verify results consistency
    results_match = np.allclose(std_solutions, vec_solutions)
    if not results_match:
        print("WARNING: Results differ between implementations!")
    
    # Print detailed results table
    print("\n============ Detailed Performance Metrics ============")
    print(f"Standard Implementation:")
    print(f"  Execution time: {std_avg_time:.2f} µs")
    print(f"  Throughput: {std_throughput_kops:.2f} KOPS ({std_throughput:.2f} samples/s)")
    print(f"  Latency jitter: ±{std_jitter:.2f} µs")
    print(f"\nVectorized Implementation:")
    print(f"  Execution time: {vec_avg_time:.2f} µs")
    print(f"  Throughput: {vec_throughput_kops:.2f} KOPS ({vec_throughput:.2f} samples/s)")
    print(f"  Latency jitter: ±{vec_jitter:.2f} µs")
    print(f"\nComparison:")
    print(f"  Speedup: {std_avg_time/vec_avg_time:.2f}x")
    print(f"  Initialization overhead: {init_time:.2f} µs")
    print("=====================================================\n")
    
    return {
        "std_exec_time": std_avg_time,
        "std_throughput": std_throughput_kops,
        "std_jitter": std_jitter,
        "vec_exec_time": vec_avg_time,
        "vec_throughput": vec_throughput_kops,
        "vec_jitter": vec_jitter,
        "init_time": init_time,
        "speedup": std_avg_time/vec_avg_time
    }


In [ ]:
def visualize_decision_boundaries(resolution=50):
    """
    Visualize the decision boundaries of the algorithm.
    Works for 2D parameter space with multiple outputs.
    
    Args:
        resolution: Number of points along each axis in the grid
    """
    # Only supports 2D parameter space
    if PDAQP_N_PARAMETER != 2:
        print(f"Visualization only supported for 2D parameter space, but N_PARAMETER = {PDAQP_N_PARAMETER}")
        return
    
    # Generate a grid of parameter values
    x = np.linspace(-3, 3, resolution)
    y = np.linspace(-3, 3, resolution)
    X, Y = np.meshgrid(x, y)
    
    # Evaluate the function at each grid point
    Z = np.zeros((resolution, resolution, PDAQP_N_SOLUTION))
    solution = np.zeros(PDAQP_N_SOLUTION)
    
    print("Generating visualization (this may take a moment)...")
    for i in range(resolution):
        for j in range(resolution):
            parameter = np.array([X[i, j], Y[i, j]])
            pdaqp_evaluate(parameter, solution)
            Z[i, j] = solution.copy()
    
    # Plot the decision boundaries
    fig, axes = plt.subplots(1, PDAQP_N_SOLUTION, figsize=(15, 5))
    
    # Handle case with only one output (axes not being iterable)
    if PDAQP_N_SOLUTION == 1:
        axes = [axes]
    
    for i in range(PDAQP_N_SOLUTION):
        im = axes[i].imshow(Z[:, :, i], extent=[-3, 3, -3, 3], origin='lower', cmap='viridis')
        axes[i].set_title(f'Solution component {{i+1}}')
        axes[i].set_xlabel('Parameter 1')
        axes[i].set_ylabel('Parameter 2')
        plt.colorbar(im, ax=axes[i])
    
    plt.tight_layout()
    plt.show()


In [ ]:
def generate_performance_summary(metrics):
    """
    Generate a performance summary table.
    
    Args:
        metrics: Dictionary of performance metrics
    """
    print("\n======= Performance Summary =======")
    print("| Implementation | Execution Time (µs) | Throughput (KOPS) | Jitter (µs) |")
    print("|----------------|--------------------:|------------------:|------------:|")
    print(f"| Standard       | {metrics['std_exec_time']:18.2f} | {metrics['std_throughput']:16.2f} | ±{metrics['std_jitter']:9.2f} |")
    print(f"| Vectorized     | {metrics['vec_exec_time']:18.2f} | {metrics['vec_throughput']:16.2f} | ±{metrics['vec_jitter']:9.2f} |")
    print(f"| **Speedup**    | {metrics['speedup']:17.2f}x | {metrics['speedup']:15.2f}x |             |")
    print("\nNotes:")
    print("- KOPS = Thousands of samples processed per second")
    print("- Jitter = Standard deviation of execution times")
    print("- All measurements performed on CPU")


In [ ]:
# Main execution block
print("PDAQP Algorithm Benchmark")
print("==================================")
print(f"Implementation details:")
print(f"- Algorithm parameters: {PDAQP_N_PARAMETER} inputs, {PDAQP_N_SOLUTION} outputs")
print(f"- Standard and vectorized implementations")
print("==================================")

# Run basic benchmarks
std_time, vec_time = benchmark_and_compare()

# Run extended benchmarks
cpu_metrics = extended_benchmark()

# Generate performance summary
generate_performance_summary(cpu_metrics)


In [ ]:
# Generate visualization of decision boundaries
if PDAQP_N_PARAMETER == 2:  # Only for 2D parameter space
    print("\nGenerating decision boundary visualization...")
    visualize_decision_boundaries(resolution=50)
else:
    print("\nVisualization not available for parameter dimension > 2")
